In [73]:
# Import
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
import warnings

In [108]:
fn_en = '/mnt/permanent/home/eszti/dipterv/panlex/data/smith/smith_all_data/en_orig.txt'
fn_it = '/mnt/permanent/home/eszti/dipterv/panlex/data/smith/smith_all_data/it.txt'
limit = None
encoding = 'latin-1'

In [75]:
class EmbeddingModel():
    def __init__(self):
        self.syn0 = None
        self.index2word = None

    def normalize(self):
        self.syn0 /= np.sqrt((self.syn0 ** 2).sum(1))[:, None]
        print('normalized, # > 0.001: {}'.format(
            len(np.where(abs(np.linalg.norm(self.syn0, axis=1) - 1) > 0.0001)[0]) ))

    def _read(self, fn, limit=None, lexicon=None, encoding='utf-8'):
        raise NotImplementedError

    def read(self, fn, limit=None, lexicon=None, encoding='utf-8'):
        print('Reading embedding from {}'.format(fn))
        self._read(fn, limit, lexicon, encoding)
        self.normalize()
        print('Syn0 size: {0}'.format(self.syn0.shape))

    def get(self, word):
        if word not in self.index2word:
            raise ValueError('Out of dictionary word: {}'.format(word))
        else:
            idx = self.index2word.index(word)
            return self.syn0[idx]

class TextEmbedding(EmbeddingModel):
    def __init__(self):
        EmbeddingModel.__init__(self)

    def _read(self, fn, limit=None, lexicon=None, encoding='utf-8'):
        id2row = []
        def filter_lines(f):
            for i,line in enumerate(f):
                if limit is not None and i > limit:
                    break
                word = line.split()[0]
                if i != 0 and (lexicon is None or word in lexicon):
                    id2row.append(word)
                    yield line

        #get the number of columns
        with open(fn, encoding=encoding) as f:
            f.readline()
            ncols = len(f.readline().split())

        with open(fn, encoding=encoding) as f:
            m = np.matrix(np.loadtxt(filter_lines(f),
                          comments=None, usecols=range(1,ncols)))
        self.syn0 = np.asarray(m)
        self.index2word = id2row

In [76]:
emb_mod_it = TextEmbedding()
emb_mod_it.read(fn=fn_it, limit=limit, encoding=encoding)

Reading embedding from /mnt/permanent/home/eszti/dipterv/panlex/data/smith/smith_all_data/it.txt
normalized, # > 0.001: 0
Syn0 size: (200000, 300)


In [109]:
emb_mod_en = TextEmbedding()
emb_mod_en.read(fn=fn_en, limit=limit, encoding=encoding)

Reading embedding from /mnt/permanent/home/eszti/dipterv/panlex/data/smith/smith_all_data/en_orig.txt


/home/eszti/.virtualenvs/dipterv3/lib/python3.5/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


normalized, # > 0.001: 0
Syn0 size: (200000, 300)


/home/eszti/.virtualenvs/dipterv3/lib/python3.5/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in greater
  if __name__ == '__main__':


In [136]:
def print_num(num):
    print(emb_mod_en.index2word[num])
    print(emb_mod_en.syn0[num])
    
def check_nans(emb):
    nans = np.where(np.isnan(emb.syn0).any(axis=1))[0]
    print(len(nans))
    print(nans)
    for nan in nans:
        print_num(nan)

In [137]:
check_nans(emb_mod_en)

5
[ 32150 122170 140745 141044 154961]
----------------------------------------------------------------------------------------------------
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  na

In [138]:
check_nans(emb_mod_it)

0
[]
